# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [1]:
!pip install yelpapi
!pip install tqdm

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
pymysql.install_as_MySQLdb()
from urllib.parse import quote_plus as urlquote

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [4]:
# Load API Credentials
with open (r"\Users\amber\.secret\yelp_api.json") as f:
    login = json.load(f)

In [5]:
# Instantiate YelpAPI Variable
yelp = YelpAPI(login['API Key'], timeout_s = 5.0)

### Define Search Terms and File Paths

In [6]:
# set our API call parameters and filename before the first call
location = 'Seattle, WA 98101'
term = 'sushi'

In [7]:
## Specify fodler for saving data
FOLDER = 'Data/'

os.makedirs(FOLDER, exist_ok = True)

# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

### Check if Json File exists and Create it if it doesn't

In [10]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder) > 0:
        # create the folder
        os.makedirs(folder, exist_ok = True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE, 'w') as f:
        json.dump([], f)
        
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/Seattle-sushi.json not found. Saving empty list to file.


In [11]:
os.path.isfile(JSON_FILE)

True

### Load JSON FIle and account for previous results

### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [13]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp.search_query(term = term, location = location)
type(results)

dict

In [14]:
len(results)

3

In [15]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [16]:
## How many results total?
results['total']

210

In [17]:
results['region']

{'center': {'longitude': -122.33405113220215, 'latitude': 47.611428139373075}}

In [18]:
results['businesses']

[{'id': '3hxncVCRPfmKLOyiXKXbDw',
  'alias': 'sushi-kashiba-seattle',
  'name': 'Sushi Kashiba',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/kDUe-mbgXOzne_HSbKy19g/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/sushi-kashiba-seattle?adjust_creative=jh5hXS_D2zcYEgEagqv7NQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=jh5hXS_D2zcYEgEagqv7NQ',
  'review_count': 1381,
  'categories': [{'alias': 'japanese', 'title': 'Japanese'},
   {'alias': 'sushi', 'title': 'Sushi Bars'}],
  'rating': 4.5,
  'coordinates': {'latitude': 47.6099251, 'longitude': -122.3415442},
  'transactions': ['delivery'],
  'price': '$$$$',
  'location': {'address1': '86 Pine St',
   'address2': 'Ste 1',
   'address3': '',
   'city': 'Seattle',
   'zip_code': '98101',
   'country': 'US',
   'state': 'WA',
   'display_address': ['86 Pine St', 'Ste 1', 'Seattle, WA 98101']},
  'phone': '+12064418844',
  'display_phone': '(206) 441-8844',
  'distance': 586.0441840262457}

- Where is the actual data we want to save?

In [20]:
## How many results total?
pd.DataFrame(results['businesses'])

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,3hxncVCRPfmKLOyiXKXbDw,sushi-kashiba-seattle,Sushi Kashiba,https://s3-media2.fl.yelpcdn.com/bphoto/kDUe-m...,False,https://www.yelp.com/biz/sushi-kashiba-seattle...,1381,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 47.6099251, 'longitude': -122.341...",[delivery],$$$$,"{'address1': '86 Pine St', 'address2': 'Ste 1'...",+12064418844,(206) 441-8844,586.044184
1,L8RRAd-JZ0Bd4MER0yyX-g,japonessa-sushi-cocina-seattle,Japonessa Sushi Cocina,https://s3-media3.fl.yelpcdn.com/bphoto/vucCrk...,False,https://www.yelp.com/biz/japonessa-sushi-cocin...,5251,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 47.6079793649921, 'longitude': -1...",[delivery],$$,"{'address1': '1400 1st Ave', 'address2': '', '...",+12069717979,(206) 971-7979,535.782040
2,YSQiqH7RIWORk_Qp-A4SOg,shiros-seattle,Shiro's,https://s3-media1.fl.yelpcdn.com/bphoto/Gs9xeF...,False,https://www.yelp.com/biz/shiros-seattle?adjust...,2584,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 47.6147782162181, 'longitude': -1...","[delivery, pickup]",$$$$,"{'address1': '2401 2nd Ave', 'address2': '', '...",+12064439844,(206) 443-9844,1062.601123
3,vs5dhED5Yf8kqPp06OD2QQ,momiji-seattle,Momiji,https://s3-media1.fl.yelpcdn.com/bphoto/oQgt6B...,False,https://www.yelp.com/biz/momiji-seattle?adjust...,1797,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 47.61482, 'longitude': -122.31657}",[delivery],$$,"{'address1': '1522 12th Ave', 'address2': '', ...",+12064574068,(206) 457-4068,1366.179557
4,EWrDx-M8HJpR0wVJHrflNg,umi-sake-house-seattle,Umi Sake House,https://s3-media3.fl.yelpcdn.com/bphoto/aNbCmx...,False,https://www.yelp.com/biz/umi-sake-house-seattl...,4038,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 47.61339, 'longitude': -122.34597}",[delivery],$$$,"{'address1': '2230 1st Ave', 'address2': None,...",+12063748717,(206) 374-8717,924.870199
5,CwvRIblqSTc1McIE9hwhFA,sushi-kudasai-seattle-2,Sushi Kudasai,https://s3-media2.fl.yelpcdn.com/bphoto/fXSJAz...,False,https://www.yelp.com/biz/sushi-kudasai-seattle...,145,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 47.610282, 'longitude': -122.333958}","[delivery, pickup]",$$,"{'address1': '1420 5th Ave', 'address2': '', '...",+12066828000,(206) 682-8000,127.635824
6,88I5mTFd8FiW3DW5S6DHQg,noren-sushi-kitchen-and-bar-seattle-2,Noren Sushi Kitchen & Bar,https://s3-media3.fl.yelpcdn.com/bphoto/aI1jq8...,False,https://www.yelp.com/biz/noren-sushi-kitchen-a...,212,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 47.614225, 'longitude': -122.327007}","[delivery, pickup]",$$,"{'address1': '320 E Pike St', 'address2': '', ...",+12063279269,(206) 327-9269,612.810300
7,ClAQfNyMp_EuTks0fz-KXg,aburiya-bento-house-seattle-2,Aburiya Bento House,https://s3-media4.fl.yelpcdn.com/bphoto/ySyh0E...,False,https://www.yelp.com/biz/aburiya-bento-house-s...,402,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 47.61134, 'longitude': -122.34493}","[delivery, pickup]",$$,"{'address1': '2100 Western Ave', 'address2': '...",+12064534685,(206) 453-4685,806.791837
8,OAefcMo0Oq2eSoK2F0ZCOg,sugo-hand-roll-bar-seattle-2,Sugo Hand Roll Bar,https://s3-media4.fl.yelpcdn.com/bphoto/lHtYgQ...,False,https://www.yelp.com/biz/sugo-hand-roll-bar-se...,403,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 47.61053, 'longitude': -122.344383}",[restaurant_reservation],NaN,"{'address1': '2001 Western Ave', 'address2': '...",+12062680172,(206) 268-0172,778.188897
9,dkZiHbwggWsB3QSQhy2New,rondo-seattle-2,Rondo,https://s3-media2.fl.yelpcdn.com/bphoto/8IgsAS...,False,https://www.yelp.com/biz/rondo-seattle-2?adjus...,494,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 47.62082, 'longitude': -122.32066}","[delivery, pickup]",$$,"{'addres

In [21]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [22]:
(results['total'])/results_per_page

10.5

In [23]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total'])/results_per_page)
n_pages

11

In [25]:
for i in tqdm_notebook( range(1,n_pages+1)):
    ## The block of code we want to TRY to run
    try:
        time.sleep(.2)
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
        
        ## save number of results for to use as offset
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp.search_query(location = location, term = term,
                                   offset = n_results+1)

        ## append new results and save to file
        previous_results.extend(results['businesses'])
        with open (JSON_FILE, 'w') as f:
            json.dump(previous_results, f)

            
    ## What to do if we get an error/exception.
    except Exception as e:
        print(' [!] ERROR', e)


  0%|          | 0/11 [00:00<?, ?it/s]

## Open the Final JSON File with Pandas

In [27]:
df = pd.read_json(JSON_FILE)

In [28]:
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,L8RRAd-JZ0Bd4MER0yyX-g,japonessa-sushi-cocina-seattle,Japonessa Sushi Cocina,https://s3-media3.fl.yelpcdn.com/bphoto/vucCrk...,False,https://www.yelp.com/biz/japonessa-sushi-cocin...,5251,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 47.6079793649921, 'longitude': -1...",[delivery],$$,"{'address1': '1400 1st Ave', 'address2': '', '...",+12069717979,(206) 971-7979,535.782040
1,YSQiqH7RIWORk_Qp-A4SOg,shiros-seattle,Shiro's,https://s3-media1.fl.yelpcdn.com/bphoto/Gs9xeF...,False,https://www.yelp.com/biz/shiros-seattle?adjust...,2585,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 47.6147782162181, 'longitude': -1...","[pickup, delivery]",$$$$,"{'address1': '2401 2nd Ave', 'address2': '', '...",+12064439844,(206) 443-9844,1062.601123
2,vs5dhED5Yf8kqPp06OD2QQ,momiji-seattle,Momiji,https://s3-media1.fl.yelpcdn.com/bphoto/oQgt6B...,False,https://www.yelp.com/biz/momiji-seattle?adjust...,1797,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 47.61482, 'longitude': -122.31657}",[delivery],$$,"{'address1': '1522 12th Ave', 'address2': '', ...",+12064574068,(206) 457-4068,1366.179557
3,EWrDx-M8HJpR0wVJHrflNg,umi-sake-house-seattle,Umi Sake House,https://s3-media3.fl.yelpcdn.com/bphoto/aNbCmx...,False,https://www.yelp.com/biz/umi-sake-house-seattl...,4038,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 47.61339, 'longitude': -122.34597}",[delivery],$$$,"{'address1': '2230 1st Ave', 'address2': None,...",+12063748717,(206) 374-8717,924.870199
4,CwvRIblqSTc1McIE9hwhFA,sushi-kudasai-seattle-2,Sushi Kudasai,https://s3-media2.fl.yelpcdn.com/bphoto/fXSJAz...,False,https://www.yelp.com/biz/sushi-kudasai-seattle...,145,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 47.610282, 'longitude': -122.333958}","[pickup, delivery]",$$,"{'address1': '1420 5th Ave', 'address2': '', '...",+12066828000,(206) 682-8000,127.635824


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             209 non-null    object 
 1   alias          209 non-null    object 
 2   name           209 non-null    object 
 3   image_url      209 non-null    object 
 4   is_closed      209 non-null    bool   
 5   url            209 non-null    object 
 6   review_count   209 non-null    int64  
 7   categories     209 non-null    object 
 8   rating         209 non-null    float64
 9   coordinates    209 non-null    object 
 10  transactions   209 non-null    object 
 11  price          187 non-null    object 
 12  location       209 non-null    object 
 13  phone          209 non-null    object 
 14  display_phone  209 non-null    object 
 15  distance       209 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 24.8+ KB


In [30]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Seattle-sushi.csv.gz'

In [31]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression = 'gzip', index = False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [32]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 202,800 Bytes
CSV.GZ FILE: 28,865 Bytes
the csv.gz is 7.025809804261216 times smaller!


## Next Class: Processing the Results and Mapping 